In [1]:
import sys
sys.path.append('../')

In [2]:
import torch
import torch.nn.functional as F
import soundfile

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

from clasp import PLCLASP
from datamodule.utils import get_log_melspec
from text.tokeniser import Tokeniser
from torch.nn.utils.rnn import pad_sequence


/fsx/home-knoriy/miniconda3/envs/clasp_2/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
model = PLCLASP.load_from_checkpoint('/fsx/knoriy/code/CLASP/.models/BAD-perciver-SimTran.ckpt').to(device)
tokeniser = Tokeniser()

### Create mel and tokenise text

In [6]:
########
# Set path to audio file
########
audio_path = './hello_world.mp3'
text = ["hello world", "how are you doing today", "I am doing great"]

########
# Load audio and text
########
audio, sample_rate = soundfile.read(audio_path)
mel = get_log_melspec(audio, sample_rate).permute(1,0).unsqueeze(0)

label = pad_sequence([torch.tensor(tokeniser.encode(t, 'en')) for t in text]).T

## Inference

In [9]:
model.freeze()
with torch.no_grad():
    label = label.to(device)
    mel = mel.to(device)

    ###############
    # Audio Features
    ###############
    audio_features = model.encode_audio(mel)
    audio_features = F.normalize(audio_features, dim=-1)
    audio_features = model.model.audio_transform(audio_features)

    ###############
    # Text Features
    ###############
    text_features = model.encode_text(label)
    text_features = F.normalize(text_features, dim=-1)
    text_features = model.model.text_transform(text_features)

    ###############
    # Get Temps
    ###############
    text_temp, audio_temp = model.get_temps()

    ###############
    # logits
    ###############
    logits_per_audio = (audio_temp * (audio_features @ text_features.T))
    # logits_per_audio = (audio_temp * (audio_features @ zeroshot_weights.T))

    ###############
    # Get metrics
    ###############
    print(text[torch.argmax(logits_per_audio)])

how are you doing today
